In [1]:
import torch
from sklearn.linear_model import LinearRegression
import numpy as np
import os
import string

'''
MIPS contains three regression methods: lienar regression, boolean regression and symbolic regression.
This file implments linear regression (LR).
Taking in integer data table (output from integer autoencoder), SR aims to try if they have simple linear relations.
'''

def load_tensors(task):
    # Construct the file paths
    a_path = f"./tasks/{task}/A_best.pt"
    b_path = f"./tasks/{task}/b_best.pt"
    z_path = f"./tasks/{task}/Z_best.pt"
    z2_path = f"./tasks/{task}/Z2_best.pt"
    hidden = f"./tasks/{task}/hidden.pt"
    hidden2 = f"./tasks/{task}/hidden2.pt"

    # Load the tensors
    A = torch.load(a_path)
    b = torch.load(b_path)
    Z = torch.load(z_path)
    Z2 = torch.load(z2_path)

    hidden = torch.load(hidden)
    hidden2 = torch.load(hidden2)
    return A, b, Z, Z2, hidden, hidden2

def get_data(task_name, batch=1000):
    A, b, Z, Z2, hidden, hidden2 = load_tensors(task_name)
    Z = np.round(Z); Z2 = np.round(Z2)
    data = torch.load(f"../tasks/{task_name}/data.pt")
    inputs = data[0].detach().numpy()
    outputs = data[1].detach().numpy()

    if len(data[0].shape) == 2:
        inputs_last = data[0][:,[-1]].detach().numpy()
        input_dim_larger_than_one_flag = False
    else:
        input_dim = data[0].shape[2]
        if input_dim > 1:
            input_dim_larger_than_one_flag = True
        else:
            input_dim_larger_than_one_flag = False
        inputs_last = data[0][:,-1].detach().numpy()
    outputs_last = data[1][:,[-1]].detach().numpy()
    if len(outputs_last.shape) == 3:
        outputs_last = outputs_last[:,:,0]
    if batch == None:
        return A, b, Z, Z2, hidden, hidden2, inputs, inputs_last, outputs, outputs_last, input_dim_larger_than_one_flag
    else:
        return A, b, Z[:batch], Z2[:batch], hidden[:batch], hidden2[:batch], inputs[:batch], inputs_last[:batch], outputs[:batch], outputs_last[:batch], input_dim_larger_than_one_flag

def produce_program_linear_regression(task_name, print_code=False, h_round=None):
    
    def linear(input, output, show_eqn=True, rounding=True):

        reg = LinearRegression().fit(input, output)
        score = reg.score(input, output)
        coeff = reg.coef_
        intercept = reg.intercept_

        if show_eqn:
            if rounding == False:
                equation = "y = " + " + ".join([f"{coef}*x{idx}" for idx, coef in enumerate(coeff)]) + f" + {intercept}"
            else:
                equation = "y = " + " + ".join([f"{np.round(coef).astype(int)}*x{idx}" for idx, coef in enumerate(coeff)]) + f" + {np.round(intercept).astype(int)}"
            #print(f"Linear Equation:{equation}")

        if rounding == False:
            return score, coeff, intercept, reg
        else:
            return score, np.round(coeff).astype(int), np.round(intercept).astype(int), reg

    def polynomial_fit(input, output, degree=2, show_eqn=True, rounding=True):
        poly = PolynomialFeatures(degree=degree)
        input_poly = poly.fit_transform(input)

        reg = LinearRegression().fit(input_poly, output)
        score = reg.score(input_poly, output)

        if show_eqn:
            # Formatting coefficients with corresponding powers
            if rounding == False:
                equation_terms = [f"{coef}*x^{i}" for i, coef in enumerate(reg.coef_)]
            else:
                equation_terms = [f"{np.round(coef)}*x^{i}" for i, coef in enumerate(reg.coef_)]
            equation = "y = " + " + ".join(equation_terms) + f" + {reg.intercept_}"
            print(f"Polynomial Equation (degree {degree}): {equation}")

        if rounding == False:
            return score, reg.coef_, reg.intercept_, reg
        else:
            print(np.round(reg.coef_))
            return score, np.round(reg.coef_), np.round(reg.intercept_), reg


    data = get_data(task_name)
    A, b, Z, Z2, hidden, hidden2, inputs, inputs_last, outputs, outputs_last, input_dim_larger_than_one_flag = data
    
    print("hidden_dim={}".format(Z.shape[1]))
    
    if input_dim_larger_than_one_flag == True:
        print("input_dim > 1, skip...")
        code = "success_flag = 0 # input_dim > 1 for linear regression"
        with open(f'./programs/{task}.txt'.replace('_','-'),"w") as f:
            f.writelines(code)
        return code, Z.shape[1]
    
    # linear regression (input, hidden) => hidden
    combine_input_Z2 = np.concatenate([inputs_last, Z2], axis=1)
    _, coeff_h, intercept_h, _ = linear(combine_input_Z2, Z)
    coeff_i2h, coeff_h2h = coeff_h[:,:inputs_last.shape[1]], coeff_h[:,inputs_last.shape[1]:]
    
    # linear regression hidden => output
    
    _, coeff_o, intercept_o, _ = linear(Z, outputs_last)
    
    num_seq = inputs.shape[1]
    num_data = inputs_last.shape[0]
    dim_hidden = Z.shape[1]

    # note that here hidden is not zero! it should be translated to lattice coordinate
    # Ah + b = 0
    if h_round == None:
        # hidden should be initialized to be the 
        h = - np.round(np.matmul(b, np.linalg.inv(A))).astype('int')
    else:
        for i in range(dim_hidden):
            h = - np.matmul(b, np.linalg.inv(A))
            if len(h.shape) == 2:
                h = h[0]
            if h_round[i] == '+':
                h[i] = np.ceil(h[i]).astype('int')
            else:
                h[i] = np.floor(h[i]).astype('int')
    
    variables = string.ascii_lowercase[:14] # hidden variables denoted as a,b,c,d,...; input variable denoted as x; output variable denoted as y.
    # initialize_code (initialize hidden states)
    initialize_code = ""
    if len(h.shape) == 2:
        h = h[0]
    for i in range(dim_hidden):
        initialize_code += f"{variables[i]} = {h[i]};"

    # hidden code
    hidden_code = ""
    
    for i in range(dim_hidden):
        hidden_i_code = f"next_{variables[i]} = "
        len_start = len(hidden_i_code)
        j0 = 0
        for j in range(dim_hidden):
            if coeff_h2h[i,j] == 0:
                j0 -= 1
            elif coeff_h2h[i,j] == 1:
                if j0 == 0:
                    hidden_i_code += f"{variables[j]}"
                else:
                    hidden_i_code += f"+{variables[j]}"
            elif coeff_h2h[i,j] == -1:
                hidden_i_code += f"-{variables[j]}"
            elif coeff_h2h[i,j] > 1:
                if j0 == 0:
                    hidden_i_code += f"{coeff_h2h[i,j]}*{variables[j]}"
                else:
                    hidden_i_code += f"+{coeff_h2h[i,j]}*{variables[j]}"
            elif coeff_h2h[i,j] < -1:
                hidden_i_code += f"{coeff_h2h[i,j]}*{variables[j]}"
                
            j0 += 1
        
        if coeff_i2h[i,0] == 0:
            pass
        elif coeff_i2h[i,0] == 1:
            hidden_i_code += f"+x"
        elif coeff_i2h[i,0] == -1:
            hidden_i_code += f"-x"
        elif coeff_i2h[i,0] > 1:
            hidden_i_code += f"+{coeff_i2h[i,0]}*x"
        elif coeff_i2h[i,0] < -1:
            hidden_i_code += f"{coeff_i2h[i,0]}*x"
        
        if intercept_h[i] == 0:
            pass
        elif intercept_h[i] > 0:
            hidden_i_code += f"+{intercept_h[i]}"
        else:
            hidden_i_code += f"{intercept_h[i]}"
            
        if len(hidden_i_code) == len_start:
            hidden_i_code += "0"
            
        if i == 0:
            hidden_code = hidden_i_code
        else:
            hidden_code += "\n        " + hidden_i_code
            
    hidden_code += "\n        "
            
    for i in range(dim_hidden):
        hidden_code += f"{variables[i]} = next_{variables[i]};"
            
    
    # output code
    output_code = f"y = "
    len_start = len(output_code)
    j0 = 0
    for j in range(dim_hidden):
        if coeff_o[0,j] == 0:
            j0 -= 1
        elif coeff_o[0,j] == 1:
            if j == 0:
                output_code += f"{variables[j]}"
            else:
                output_code += f"+{variables[j]}"
        elif coeff_o[0,j] == -1:
            output_code += f"-{variables[j]}"
        elif coeff_o[0,j] > 1:
            if j == 0:
                output_code += f"{coeff_o[0,j]}*{variables[j]}"
            else:
                output_code += f"+{coeff_o[0,j]}*{variables[j]}"
        elif coeff_o[0,j] < -1:
            output_code += f"{coeff_o[0,j]}*{variables[j]}"
        j0 += 1
            
    if intercept_o[0] == 0:
        pass
    elif intercept_o[0] > 0:
        output_code += f"+{intercept_o[0]}"
    else:
        output_code += f"{intercept_o[0]}"
        
    if len(output_code) == len_start:
        output_code += "0"
    
    
    function_code = f"""
def f(s):
    {initialize_code}
    ys = []
    for i in range({num_seq}):
        x = s[i]
        {hidden_code}
        {output_code}
        ys.append(y)
    return ys
    """
    
    preprocess_code = f"""
import numpy as np
data = get_data(\"{task_name}\")
A, b, Z, Z2, hidden, hidden2, inputs, inputs_last, outputs, outputs_last, _ = data
num_example = inputs.shape[0]
    """
    
    verify_code = f"""
def verify():
    wrong = 0
    for i in range(int(num_example*0.01)):
    #for i in range(int(num_example)):
        out_pred = f(inputs[i])
        wrong += np.sum(1 - (np.array(outputs[i]) == np.array(out_pred)))

    if wrong == 0:
        success_flag = 1
        print('verification success')
    else:
        success_flag = 0
        print('verification failure')

    return success_flag
    
success_flag = verify()
"""
    
    code = f"""
{preprocess_code}
{function_code}
{verify_code}
"""
    if print_code == True:
        print(code)
        
    with open(f'./programs/{task}.txt'.replace('_','-'),"w") as f:
        f.writelines(function_code)
    return code, dim_hidden
    